#### Name: Reda Mohsen Reda
#### ID: 18P5141
#### Course: Deep Learning
#### Topic: Hand Gesture Recognition Project

# Instal Dependencies

In [1]:
# !pip install tensorflow opencv-python matplotlib ipympl

# Import Libraries and Setup GPU

In [ ]:
import os
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import random

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_physical_devices('GPU')

In [ ]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dropout, Dense


# About Data

[`hand-gesture-recognition-dataset`](https://www.kaggle.com/datasets/aryarishabh/hand-gesture-recognition-dataset?resource=download) This dataset contains total 24000 images of 20 different gestures. For training purpose, there are 900 images in each directory and for testing purpose there are 300 images in each directory.

![Full dataset](Images/dataset-cover.png)

# Load Data

 Loading the data into training dataset and testing dataset using tensorflow [`utils`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory)

In [ ]:
batch_size = 32
img_height = 50
img_width = 50
class_names = ['0','1','2','3','4','5','6','7','8','9',
               '10','11','12','13','14','15','16','17',
               '18','19']

train_dataset = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    labels='inferred',
    # label_mode='categorical',
    label_mode='int',
    color_mode='grayscale',
    class_names=class_names,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=1,
    validation_split=0.33335,
    subset='training'
)

valid_dataset = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    labels='inferred',
    # label_mode='categorical',
    label_mode='int',
    color_mode='grayscale',
    class_names=class_names,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=1,
    validation_split=0.33335,
    subset='validation'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    'dataset/test',
    labels='inferred',
    # label_mode='categorical',
    label_mode='int',
    color_mode='grayscale',
    class_names=class_names,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True
)

if label_mode is categorical, the labels are a float32 tensor of shape (batch_size, num_classes), representing a one-hot encoding of the class index.
if color_mode is grayscale, there's 1 channel in the image tensors.

In [ ]:
train_dataset.take(1)

# Normalize Data

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
normalized_train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
normalized_valid_dataset = valid_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_train_dataset))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

# Visualize Data

In [ ]:
print('Images Shape', image_batch.shape)
print('Labels Shape', labels_batch.shape)
plt.imshow(image_batch[0],cmap='gray')
plt.title(class_names[labels_batch[0]])

In [ ]:
plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i],cmap='gray')
    # plt.title(class_names[labels_batch[i]])
    plt.title(labels_batch[i])
    plt.axis("off")

# Configure the dataset for performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = normalized_train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
valid_dataset = normalized_valid_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Build Deep Learning Model

In [ ]:
model = Sequential()
# Layer 1
model.add(Conv2D(filters=6, input_shape=(50,50,1), kernel_size=5, strides=1, activation='relu',padding="same"))
# Layer 2
model.add(AveragePooling2D(strides=2))
# Layer 3
model.add(Conv2D(filters=16 ,kernel_size=5, strides=1, activation='relu'))
# Layer 4
model.add(AveragePooling2D(strides=2))
# Layer 5
model.add(Conv2D(filters=120,kernel_size=5,strides=1, activation='relu'))
# Flatten
model.add(Flatten())
# Layer 6
model.add(Dense(120, activation='relu'))
# Dropout
model.add(Dropout(0.4))
# Layer 7
model.add(Dense(84, activation='relu'))
# Output Layer
model.add(Dense(20, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train

In [ ]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# import time
# class CustomCallback(keras.callbacks.Callback):
#     def on_train_begin(self, logs=None):
#         self.now= time.time()
#     def on_train_end(self,totaltime, logs=None):
#         later=time.time()
#         duration=later-self.now 
#         totaltime = duration
#         print('Total Time is: ', totaltime, ' seconds')

In [ ]:
epochs=50
# hist = model.fit(train_dataset, validation_data=valid_dataset, epochs=epochs, batch_size=batch_size, callbacks=[CustomCallback(),tensorboard_callback])
epochs=50
hist = model.fit(train_dataset, validation_data=valid_dataset, epochs=epochs, batch_size=batch_size, callbacks=[tensorboard_callback])

# Plot Performance

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Evaluate

In [ ]:
print('\nEvaluating:')
(test_loss, test_accuracy) = model.evaluate(test_dataset)
print(f'\nTest accuracy: {test_accuracy * 100:>0.1f}%, Test loss: {test_loss:>8f}')

# Test

In [ ]:
# test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))
# # Retrieve a batch of images from the test set
# image_batch, label_batch = test_dataset.as_numpy_iterator().next()
# predictions = model.predict_on_batch(image_batch).flatten()

# # Apply a sigmoid since our model returns logits
# # predictions = tf.nn.sigmoid(predictions)
# # predictions = tf.where(predictions < 0.5, 0, 1)

# print('Predictions:\n', predictions.numpy())
# print('Labels:\n', label_batch)

# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].astype("uint8"))
#   plt.title(class_names[predictions[i]])
#   plt.axis("off")

# Confusion Matrix

# Save the Model

In [ ]:
model.save('outputs/model')

In [ ]:
# image_batch, labels_batch = next(iter(test_dataset))
# fig = plt.figure(figsize = (20,20))
# counter = 1
# for image in image_batch[0:9]:
#     fig.add_subplot(3,3,counter)
#     pred = np.argmax(model.predict(image))
#     plt.imshow(image,cmap="gray")
#     plt.title("Repd"+str(pred)+"True"+str(label))
#     counter +=1

In [ ]:
# True Label of image => test.label() , predicted